In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df1 = pd.read_excel('./data/NAVER_시세추이.xlsx')

C:\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [4]:
df_atr

,date,fin,high,low
0,2021/08/31,439000,439500,427000
1,2021/08/30,428000,429000,423000
2,2021/08/27,420000,423500,416500
3,2021/08/26,423500,430500,420000
4,2021/08/25,430500,439000,427000
...,...,...,...,...
2460,2011/09/06,216000,222500,210000
2461,2011/09/05,212500,217000,206500
2462,2011/09/02,210500,218000,205000
2463,2011/09/01,207500,217500,206000


In [5]:
lst = []
atr = []

for i in range(0,len(df_atr)-1):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i+1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i+1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

In [6]:
# y값(종가 데이터) 준비

fin = list(df_atr[0:-1]['fin'])
del fin[715:718] # 거래정지 시점 데이터 행 삭제

In [7]:
# 거래량 데이터 추출

df1 = df1.drop(len(df1)-1,axis=0)
df1 = df1[['일자', '거래량']]

In [8]:
# PER, PBR 추출

df2 = pd.read_excel('./data/NAVER_PER.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [9]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/NAVER_거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

In [10]:
# 데이터 병합
NAVER = pd.merge(df1, df2, on='일자', how='outer')
NAVER = pd.merge(NAVER, df3, on='일자', how='outer')
NAVER['ATR'] = atr

In [12]:
# 액면분할에 따른 전처리
NAVER = NAVER.drop(index = [715,716,717], axis=1)
NAVER.iloc[715:,8] = NAVER.iloc[715:,8]/5
NAVER.iloc[715:,1] = NAVER.iloc[715:,1]*5

In [13]:
# 날짜 데이터로 변환
NAVER['일자'] = pd.to_datetime(NAVER['일자'])
NAVER.rename(columns={'일자':'DATE'}, inplace=True)

In [14]:
# 거시경제 지표와 병합
macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_NAVER = pd.merge(NAVER, macro_data, how='left', on='DATE')

In [15]:
df_NAVER

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y
0,2021-08-31,768823,63.84,9.79,-3.189151e+10,2.904930e+08,-7.503393e+10,1.066349e+11,12500.0,"15,259.20","4,522.68",16.48,"1,158.73",110.32,134.05
1,2021-08-30,399336,62.24,9.54,1.482470e+09,1.669520e+08,-4.377661e+10,4.212719e+10,9000.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13
2,2021-08-27,567004,61.07,9.36,1.238749e+10,4.621300e+07,3.462811e+10,-4.706182e+10,7000.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89
3,2021-08-26,645901,61.58,9.44,-9.737998e+09,5.607250e+08,6.857878e+10,-5.940151e+10,10500.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59
4,2021-08-25,677224,62.60,9.60,-4.208689e+09,5.866996e+09,6.189652e+10,-6.355482e+10,12000.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,2011-09-07,1408170,19.91,4.92,2.188560e+10,4.067550e+08,-2.704334e+09,-1.958802e+10,1300.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89
2457,2011-09-06,2454130,19.59,4.84,6.313876e+10,4.848330e+08,-1.741551e+10,-4.620809e+10,2500.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22
2458,2011-09-05,1203915,19.28,4.76,2.466103e+10,1.225445e+08,-3.525800e+08,-2.443100e+10,2100.0,NaN,NaN,NaN,NaN,NaN,NaN
2459,2011-09-02,1339225,19.09,4.72,2.678296e+10,5.359510e+08,-8.318674e+09,-1.900023e+10,2600.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30


In [16]:
# y값(종가 데이터) 병합
df_NAVER['y'] = fin

In [17]:
# 종가 열 액면분할 반영
df_NAVER.iloc[715:,15] = df_NAVER.iloc[715:,15]/5

In [18]:
# 결측치를 전일 값으로 대체
df_NAVER.sort_values('DATE', inplace=True)
df_NAVER.fillna(method='ffill', inplace=True)

In [22]:
df_NAVER.to_excel('./data/NAVER.xlsx',index=False)

In [23]:
NAVER_df = pd.read_excel('./data/NAVER.xlsx')

In [24]:
NAVER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2461 non-null   datetime64[ns]
 1   거래량            2461 non-null   int64         
 2   PER            2461 non-null   float64       
 3   PBR            2461 non-null   float64       
 4   기관 합계          2461 non-null   int64         
 5   기타법인           2461 non-null   int64         
 6   개인             2461 non-null   int64         
 7   외국인 합계         2461 non-null   int64         
 8   ATR            2461 non-null   int64         
 9   NASDAQ         2461 non-null   object        
 10  S&P            2461 non-null   object        
 11  CBOE           2461 non-null   float64       
 12  Exchange rate  2461 non-null   object        
 13  futures2y      2461 non-null   float64       
 14  futures10y     2461 non-null   float64       
 15  y              2461 n